In [1]:
import os, random
from scipy.io import wavfile
from shutil import copyfile
from sphfile import SPHFile
from WaveNetClassifier import WaveNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import h5py

Using TensorFlow backend.


In [2]:
# Cut training files to the same size and store the user ids and audio data in arrays.

user_ids = []
audio_data = []
test_user_ids = []
test_audio_data = []
# Normalize the data before saving them to arrays.

nb_bits = 16
max_nb_bit = float(2 ** (nb_bits - 1))

i = 0
j = 0
for path, dirnames, filenames in os.walk('TRAIN_WAV'):
    j = 0
    if i == 7:
        break
    for filename in filenames:         
        file_path = os.path.join(path, filename)
        fs, data = wavfile.read(file_path)
        user_id = file_path.split(os.sep)[-2]
        if j == 2:
            break
        if j == 1:
            test_user_ids.append(user_id)
            test_audio_data.append(data / (max_nb_bit + 1.0))
        else:
            user_ids.append(user_id)
            audio_data.append(data / (max_nb_bit + 1.0))
        j += 1
    i += 1
    
# Min array length
min_array = min(audio_data, key = len)
min_length = len(min_array)

for i, data in enumerate(audio_data):
    audio_data[i] = data[0 : min_length]
for i, data in enumerate(test_audio_data):
    test_audio_data[i] = data[0 : min_length]

In [3]:
# Create a vector for each user id. Categorical -> one hot encoding
# The list has to be converted the numpy array and transposed first.

X_train = np.array(audio_data)

onehot_encoder = OneHotEncoder(sparse = False, categories = 'auto')
Y_train = onehot_encoder.fit_transform(np.array(user_ids).reshape(-1, 1))

print('X_train shape:')
display(X_train.shape)

print('Y_train shape:')
display(Y_train.shape)

X_train shape:


(5, 46797)

Y_train shape:


(5, 5)

In [ ]:
wnc = WaveNetClassifier((46797,), (5,), kernel_size = 2, dilation_depth = 9, n_filters = 40, task = 'classification')

In [ ]:
wnc.fit(X_train, Y_train, epochs = 200, batch_size = 32, optimizer='adam', save=True, save_dir='results3/')